In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np

# xtest = np.load('/content/drive/MyDrive/Disk/Without_Noise/x_test256R.npz')
# xtest = xtest['arr_0']
# ytest = np.load('/content/drive/MyDrive/Disk/Without_Noise/y_test256R.npz')
# ytest = ytest['arr_0']

In [ ]:
num_images = xtest.shape[0]

In [ ]:
####

    ### Initialising the numpy arrays for each density range

####

xtest1 = np.zeros((num_images,256,256,1))    # For density range 1e16 to 1e17
xtest2 = np.zeros((num_images,256,256,1))    # For density range 1e17 to 1e18
xtest3 = np.zeros((num_images,256,256,1))    # For density range 1e18 to 1e19

ytest1 = np.zeros((num_images,256,256,1))  # For corresponding test1
ytest2 = np.zeros((num_images,256,256,1))  # For corresponding test2
ytest3 = np.zeros((num_images,256,256,1))  # For corresponding test3


In [ ]:
# erms_max = 13.32106
den_max = 1e19

In [ ]:
ytest = ytest/255.0
ytest = ytest*den_max
print(np.amax(ytest[23]))

4.5e+16


In [ ]:
# xtest = xtest/255.0
# xtest = xtest*erms_max
# print(np.amax(xtest[23]))

In [ ]:
print(xtest1_1.shape)
print(xtest2_1.shape)
print(xtest3_1.shape)

(118, 256, 256, 1)
(101, 256, 256, 1)
(111, 256, 256, 1)


In [ ]:
# indexes
i = 0
j = 0
l = 0

for k in range(num_images):
  y_test = ytest[k,:,:,:]
  
  max_test = np.amax(y_test)
  if max_test >=1e16 and max_test <1e17:
    copy = ytest[k].reshape((256,256,1))
    copy_erms = xtest[k].reshape((256,256,1))
    ytest1[i,:,:,:] = copy
    xtest1[i,:,:,:] = copy_erms
    i += 1
  elif max_test >=1e17 and max_test <1e18:
    copy2 = ytest[k].reshape((256,256,1))
    copy_erms2 = xtest[k].reshape((256,256,1))
    ytest2[j,:,:,:] = copy2
    xtest2[j,:,:,:] = copy_erms2
    j += 1
  elif max_test >=1e18 and max_test <=1e19:
    copy3 = ytest[k].reshape((256,256,1))
    copy_erms3 = xtest[k].reshape((256,256,1))
    ytest3[l,:,:,:] = copy3
    xtest3[l,:,:,:] = copy_erms3
    l += 1


ytest1_1 = np.zeros((i,256,256,1))
ytest2_1 = np.zeros((j,256,256,1))
ytest3_1 = np.zeros((l,256,256,1))
xtest1_1 = np.zeros((i,256,256,1))
xtest2_1 = np.zeros((j,256,256,1))
xtest3_1 = np.zeros((l,256,256,1))

for k in range(i):
  copy = ytest1[k].reshape((256,256,1))
  ytest1_1[k,:,:,:] = copy
  copy_x = xtest1[k].reshape((256,256,1))
  xtest1_1[k,:,:,:] = copy_x

for k in range(j):
  copy2 = ytest2[k].reshape((256,256,1))
  ytest2_1[k,:,:,:] = copy2
  copy_x2 = xtest2[k].reshape((256,256,1))
  xtest2_1[k,:,:,:] = copy_x2

for k in range(l):
  copy3 = ytest3[k].reshape((256,256,1))
  ytest3_1[k,:,:,:] = copy3
  copy_x3 = xtest3[k].reshape((256,256,1))
  xtest3_1[k,:,:,:] = copy_x3

In [ ]:
ytest1_1 = ytest1_1/den_max
ytest1_1 = ytest1_1*255.0

ytest2_1 = ytest2_1/den_max
ytest2_1 = ytest2_1*255.0

ytest3_1 = ytest3_1/den_max
ytest3_1 = ytest3_1*255.0

In [ ]:
# print(np.amax(ytest1_1))
# print(np.amax(ytest2_1))
# print(np.amax(ytest3_1))

In [ ]:
# print(np.amax(xtest1_1))
# print(np.amax(xtest2_1))
# print(np.amax(xtest3_1))

In [ ]:
### Saving the separated xtest and ytest datasets

np.savez_compressed('xtest_16_17.npz',xtest1_1)
np.savez_compressed('ytest_16_17.npz',ytest1_1)
np.savez_compressed('xtest_17_18.npz',xtest2_1)
np.savez_compressed('ytest_17_18.npz',ytest2_1)
np.savez_compressed('xtest_18_19.npz',xtest3_1)
np.savez_compressed('ytest_18_19.npz',ytest3_1)

In [2]:
# import the required libraries
import numpy as np
import pickle
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras import backend as keras
import tensorflow as tf
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import pandas as pd

In [3]:
def UNet_MCD(input_size, reg_fn, dropout_rate):
    inputs = Input(shape=input_size)

    conv1 = Conv2D(8, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(inputs)
    conv1 = Dropout(dropout_rate)(conv1, training=True)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool1)
    conv2 = Dropout(dropout_rate)(conv2, training=True)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool2)
    conv3 = Dropout(dropout_rate)(conv3, training=True)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool3)
    conv4 = Dropout(dropout_rate)(conv4, training=True)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool4)
    conv5 = Dropout(dropout_rate)(conv5, training=True)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    conv6 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool5)
    conv6 = Dropout(dropout_rate)(conv6, training=True)
    pool6 = MaxPooling2D(pool_size=(2, 2))(conv6)
      
    up7 = Conv2DTranspose(128, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(pool6)
    up7 = Dropout(dropout_rate)(up7, training=True)
    sc7 = Concatenate()([up7, pool5])

    up8 = Conv2DTranspose(64, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc7)
    up8 = Dropout(dropout_rate)(up8, training=True)
    sc8 = Concatenate()([up8, pool4])

    up9 = Conv2DTranspose(32, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc8)
    up9 = Dropout(dropout_rate)(up9, training=True)
    sc9 = Concatenate()([up9, pool3])

    up10 = Conv2DTranspose(16, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc9)
    up10 = Dropout(dropout_rate)(up10, training=True)
    sc10 = Concatenate()([up10, pool2])

    up11 = Conv2DTranspose(8, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc10)
    up11 = Dropout(dropout_rate)(up11, training=True)
    sc11 = Concatenate()([up11, pool1])

    up12 = Conv2DTranspose(1, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(sc11)
    up12 = Dropout(dropout_rate)(up12, training=True)
    
    model = Model(inputs, up12)

    model.compile(optimizer='adam', metrics=['accuracy'], loss=tf.keras.losses.MeanSquaredError())

    return model

In [ ]:
def UNet_MCD(input_size, reg_fn, dropout_rate):
    inputs = Input(shape=input_size)

    #Encoder Block 0
    conv1 = Conv2D(8, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(inputs)
    conv1 = Dropout(dropout_rate)(conv1, training=True)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    #Encoder Block 1 
    conv2 = Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool1)
    conv2 = Dropout(dropout_rate)(conv2, training=True)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    #Encoder Block 2
    conv3 = Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool2)
    conv3 = Dropout(dropout_rate)(conv3, training=True)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    #Encoder Block 3
    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool3)
    conv4 = Dropout(dropout_rate)(conv4, training=True)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    #Encoder Block 4
    conv5 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool4)
    conv5 = Dropout(dropout_rate)(conv5, training=True)
    pool5 = MaxPooling2D(pool_size=(2, 2))(conv5)

    #Encoder Block 5
    conv6 = Conv2D(256, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(pool5)
    conv6 = Dropout(dropout_rate)(conv6, training=True)
    pool6 = MaxPooling2D(pool_size=(2, 2))(conv6)

    ### Decoder Blocks

    ### Decoder Blocks

    #Decoder Block 4
    drop4_1 = Dropout(dropout_rate)(pool5, training = True)
    conv7 = Conv2D(128, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(drop4_1)
    conv7 = Dropout(dropout_rate)(conv7, training=True)
    up7 = Conv2DTranspose(128, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(conv7)
    sc7 = Concatenate()([up7, conv5])

    #Decoder Block 3
    drop3_1 = Dropout(dropout_rate)(sc7, training = True)
    conv8 = Conv2D(64, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(drop3_1)
    conv8 = Dropout(dropout_rate)(conv8, training=True)
    up8 = Conv2DTranspose(64, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(conv8)
    sc8 = Concatenate()([up8, conv4])

    #Decoder Block 2
    drop2_1 = Dropout(dropout_rate)(sc8, training = True)
    conv9 = Conv2D(32, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(drop2_1)
    conv9 = Dropout(dropout_rate)(conv9, training=True)
    up9 = Conv2DTranspose(32, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(conv9)
    sc9 = Concatenate()([up9, conv3])

    #Decoder Block 1
    conv10 = Conv2D(16, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(sc9)
    conv10 = Dropout(dropout_rate)(conv10, training=True)
    up10 = Conv2DTranspose(16, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(conv10)
    sc10 = Concatenate()([up10, conv2])

    #Decoder Block 0
    conv11 = Conv2D(8, 3, activation='relu', padding='same', kernel_regularizer=reg_fn)(sc10)
    conv11 = Dropout(dropout_rate)(conv11, training=True)
    up11 = Conv2DTranspose(8, 2, strides=(2, 2), activation='relu', padding='same', kernel_regularizer=reg_fn)(conv11)
    sc11 = Concatenate()([up11, conv1])

    conv12 = Conv2D(1, 1, activation='sigmoid')(sc11)

    model = Model(inputs, conv12)

    model.compile(optimizer='adam', metrics=['accuracy'], loss=tf.keras.losses.MeanSquaredError())

    return model


In [8]:
def run_model(lam, x_test):
    model_1 = UNet_MCD((256, 256, 1), l1(lam),0.4)
    #his = model_1.fit(x_train, y_train, epochs=300, validation_data=(x_valid, y_valid))
    #model_1.load_weights('../circular_halfdata/1e-06.h5')
    model_1.load_weights('/content/drive/MyDrive/Concentric_Rings/1e-06_MCD_P_1.h5')
    y_pred = model_1.predict(x_test)  #144x256x256x1

    # np.save(str(lam)+'_test.npz', y_t)
    #np.save(str(lam) + '_pred.npy', y_pred)
    np.savez_compressed(str(lam) + '_MCD_pred_P1_17_18.npz', y_pred)
    #np.save(str(lam) + '_acc.npy', his.history['acc'])
    #np.save(str(lam) + '_vacc.npy', his.history['val_acc'])
    #np.save(str(lam) + '_loss.npy', his.history['loss'])
    #np.save(str(lam) + '_vloss.npy', his.history['val_loss'])
    #model_1.save_weights(str(lam) + '.h5')

In [7]:
# x_test1 = np.load('/content/drive/MyDrive/Disk/Without_Noise/xtest_16_17.npz')
# x_test1 = x_test1['arr_0']
# x_test1= x_test1.astype('float32') #/ 255.0

x_test1 = np.load('/content/drive/MyDrive/Disk/Without_Noise/xtest_17_18.npz')
x_test1 = x_test1['arr_0']
x_test1 = x_test1.astype('float32') #/ 255.0

# x_test3 = np.load('/content/drive/MyDrive/Disk/Without_Noise/xtest_18_19.npz')
# x_test3 = x_test3['arr_0']
# x_test3 = x_test3.astype('float32') #/ 255.0

NameError: ignored

In [6]:
run_model(1e-6, x_test1)

4/4 [==============================] - 6s 1s/step


In [ ]:
x_test1 = np.load('/content/drive/MyDrive/Disk/Without_Noise/ytest_16_17.npz')
x_test1 = x_test1['arr_0']
x_test2 = np.load('/content/drive/MyDrive/Disk/Without_Noise/ytest_17_18.npz')
x_test2 = x_test2['arr_0']
x_test3 = np.load('/content/drive/MyDrive/Disk/Without_Noise/ytest_18_19.npz')
x_test3 = x_test3['arr_0']

In [ ]:
print(x_test1.shape[0])
print(x_test2.shape[0])
print(x_test3.shape[0])

119
111
100
